# 주제분석 2주차 패키지

## Chaper 1. Dataset

### 문제0. 기본세팅

In [1]:
import torch
import torchvision
from torchvision import transforms
import pathlib
import sklearn
import numpy as np
import PIL

### 문제1. 시드설정

In [2]:
import random
np.random.seed(17)
seed = np.random.seed(17)

### 문제2. dog, cat 변수를 만들어 폴더에 들어있는 경로들을 리스트 형태로 저장

In [3]:
from pathlib import Path
cat_dir = pathlib.Path('C:/Users/yjs49/Desktop/통계분석학회 P-SAT/패키지 방학 복습/2주차 패키지/training_set/cats')
dog_dir = pathlib.Path('C:/Users/yjs49/Desktop/통계분석학회 P-SAT/패키지 방학 복습/2주차 패키지/training_set/dogs')

In [4]:
cat = []
for i in cat_dir.iterdir():
    cat.append(i)
    
dog = []
for i in dog_dir.iterdir():
    dog.append(i)

### 문제3. 저장한 리스트를 8:2로 나누고 train/test로 구분

In [5]:
from sklearn.model_selection import train_test_split

cat_train, cat_val = train_test_split(cat, test_size = 0.2, shuffle = True, random_state = seed)
dog_train, dog_val = train_test_split(dog, test_size = 0.2, shuffle = True, random_state = seed)

In [6]:
train = cat_train + dog_train
val = cat_val + dog_val

### 문제4. torchvision.transforms를 이용하여 이미지 데이터 변화

### 4-1. train 데이터셋 변화

In [7]:
train_transforms = transforms.Compose([
    transforms.Resize((128,128)), #사이즈 조정
    transforms.RandomHorizontalFlip(), #이미지를 수직변환할 수 있게 하여 방향변화에 robust한 형태로 만들기
    transforms.ToTensor() #PIL 형태의 이미지나 ndarray 를 PyTorch 가 이해할 수 있는 tensor 자료형으로 바꾸어 주는 역할
])

### 4-2. val 데이터셋 변화

In [8]:
val_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

### 문제5. Dataset class 만들기

In [9]:
from torch.utils.data import Dataset
#데이터의 양이 방대할 경우, 데이터를 한번에 불러오기 어렵기 때문에, 하나씩 불러서 쓰는 방식을 택하기 위해 Dcustomized된 ataset을 사용함
#미니배치 학습, 데이터 셔플, 병렬 처리까지 간단하게 수행

class Dataset(Dataset) :
    
    def __init__(self, data, transforms): #생성될 때 한번만 실행됨
        self.data = data
        self.transforms = transforms
        
    def __len__(self): #데이터의 개수 
        return len(self.data)
    
    def __getitem__(self, index): #인덱스값을 기준으로 데이터셋에서 sample을 가져오고, 이를 변환하는 과정을 거침
        image_path = self.data[index]
        image = PIL.Image.open(image_path)
        image_transformed = self.transforms(image)
        
        if "cat" in str(self.data[index]) :
            label = 0
        else :
            label = 1
            
        return image_transformed, label

### 6. 만든 Dataset class를 이용하여 train, val 데이터를 Dataset 형식으로 바꾸기

In [10]:
train_dataset = Dataset(data = train, transforms = train_transforms)
val_dataset = Dataset(data = val, transforms = val_transforms)

In [11]:
print(len(train_dataset), len(val_dataset))

6404 1601


In [12]:
train_dataset[0]

(tensor([[[0.9059, 0.9059, 0.9059,  ..., 0.9255, 0.9255, 0.9255],
          [0.9059, 0.9098, 0.9020,  ..., 0.9255, 0.9255, 0.9255],
          [0.9098, 0.9137, 0.5216,  ..., 0.5176, 0.9255, 0.9255],
          ...,
          [0.9255, 0.9176, 0.8000,  ..., 0.8039, 0.9255, 0.9255],
          [0.9176, 0.9137, 0.9294,  ..., 0.9333, 0.9255, 0.9255],
          [0.9255, 0.9255, 0.9255,  ..., 0.9216, 0.9216, 0.9255]],
 
         [[0.9569, 0.9569, 0.9490,  ..., 0.9490, 0.9451, 0.9451],
          [0.9569, 0.9569, 0.9451,  ..., 0.9490, 0.9451, 0.9451],
          [0.9569, 0.9569, 0.5647,  ..., 0.5412, 0.9451, 0.9451],
          ...,
          [0.9451, 0.9373, 0.8196,  ..., 0.8275, 0.9451, 0.9451],
          [0.9373, 0.9333, 0.9490,  ..., 0.9529, 0.9451, 0.9451],
          [0.9451, 0.9451, 0.9451,  ..., 0.9412, 0.9412, 0.9451]],
 
         [[0.9294, 0.9294, 0.9255,  ..., 0.9333, 0.9216, 0.9216],
          [0.9294, 0.9294, 0.9216,  ..., 0.9333, 0.9255, 0.9216],
          [0.9294, 0.9333, 0.5412,  ...,

In [13]:
val_dataset[0]

(tensor([[[0.3216, 0.1137, 0.0353,  ..., 0.5333, 0.5294, 0.5255],
          [0.3725, 0.1647, 0.0627,  ..., 0.5333, 0.5216, 0.5255],
          [0.3843, 0.1843, 0.0627,  ..., 0.5255, 0.5176, 0.5137],
          ...,
          [0.3137, 0.3490, 0.3765,  ..., 0.1647, 0.0863, 0.0706],
          [0.2392, 0.2549, 0.2627,  ..., 0.2118, 0.1020, 0.0784],
          [0.1804, 0.1725, 0.1804,  ..., 0.2039, 0.1059, 0.0667]],
 
         [[0.2667, 0.0941, 0.0392,  ..., 0.4627, 0.4667, 0.4667],
          [0.2980, 0.1255, 0.0549,  ..., 0.4667, 0.4627, 0.4745],
          [0.2941, 0.1333, 0.0392,  ..., 0.4706, 0.4667, 0.4706],
          ...,
          [0.2510, 0.2863, 0.3176,  ..., 0.0275, 0.0275, 0.0314],
          [0.1647, 0.1765, 0.1922,  ..., 0.0471, 0.0431, 0.0392],
          [0.1098, 0.0980, 0.1098,  ..., 0.0353, 0.0471, 0.0314]],
 
         [[0.3020, 0.1176, 0.0549,  ..., 0.4902, 0.4941, 0.4941],
          [0.3333, 0.1490, 0.0706,  ..., 0.4902, 0.4824, 0.4902],
          [0.3294, 0.1529, 0.0510,  ...,

### 7. Dataset을 Dataloader로 만들기

In [14]:
from torch.utils.data import DataLoader
#batch형태로 만들어서 우리가 실제로 학습할 때 이용할 수 있게 형태를 만들어주는 라이브러리 : DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 64, shuffle = True)

In [15]:
train_dataloader

In [16]:
val_dataloader

## 2. Modelling

### 1. CNN 모델을 class 형태로 만들어주기

In [17]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() ## 기반 class를 초기화하고, 해당 class 속성을 cubclass가 스스로 받아오도록 함!
        
        ### Conv2d -> BathNorm -> ReLU -> Maxpool
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3), ###in_channels) 흑백이면 1, RGB면 3
            nn.BatchNorm2d(16),
            ### batch정규화 : 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화
            ### batch parameter의 기준은 input size
            nn.ReLU(),
            ### 양수면 자기자신, 음수면 0을 반환 / 갈수록 기울기가 소실되는 시그모이드 개선(0~1사이의 값만 내보냄)
            ### 가중치 업데이트 속도, 연산 속도가 매우 빠르고(함수가 간단), 기울기 소실(vanishing Gradient)문제가 발생하지 않음
            ### feature가 발현될 가능성을 비선형적 그래프로 나타내주는 작업이 필요(non-linearity 부여)
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            ### pooling은 파라미터를 줄여 overfitting을 방지하고, 속도를 높이기 위한 방법
            ### convolution layer를 resize하여 새로운 layer를 얻는 것, 특징을 뽑아내는 작업
        )    
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc = nn.Linear(1024,2) #평탄화 작업, Convolution/Pooling 프로세스의 결과를 취하여 다음 결과와 이어질 수 있게 연결하는 역할
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        
        x = x.reshape(-1,1024)
        
        x = self.fc(x) ### fc layer를 거친 값으로 반환
        
        return x

### 2. 그래픽카드로 장치 설정 후, 모델 그래픽 카드에 할당

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

### 3. optimizer와 loss function 정의(모델 학습을 위해 최적화, 손실 함수가 필요함!)

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #learniing rate 설정
### Adam : 모멘텀 최적화 + RMSProp / 모멘텀 최적화처럼 Gradient Descent을 따르고, RMSProp처럼 그레디언트 제곱의 지수 감소 평균을 따름

loss_function = nn.CrossEntropyLoss()
### softmax(classification score을 0~1 사이로 정규화, 모든 합이 1) + crossentropy
### 정답 class에 해당하는 값에 대해서만 log 합 진행

### 4. 2중 for문을 이용한 모델 학습, iteration 100번 지날 때마다 loss 값 출력

In [22]:
num_epoch = 30

for epoch in range(num_epoch) :
    ### enumerate : index와 원소로 이루어진 tuple 생성 (index와 원소에 동시 접근해서 loop를 돌릴 수 있음)
    for index, [image, label] in enumerate(train_dataloader) :
        x = image.to(device)
        y = label.to(device)
        
        optimizer.zero_grad()
        
        output = model.forward(x)
        
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        
        if index % 100 == 99:
            print(f'Epoch : {epoch + 1} loss: {loss : .4f}')
            
print('Finish Training')

Epoch : 1 loss:  0.2441
Epoch : 2 loss:  0.2900
Epoch : 3 loss:  0.1810


KeyboardInterrupt: 